# Docker Entrypoint with Scripts

### Introduction

So in the last lesson, we saw how Entrypoint is a Docker command that is run at runtime, and that we can pass additional arguments to at runtime.  In this lesson, we'll see how we can use Entrypoint to set up our web applications.

### An Entrypoint Usecase

Ok, so remember that what's unique about entrypoint is that, unlike CMD, it cannot be overridden.   It is always run when booting up a container.  So when working with a web application, a typical usecase would be some initial setup.  

For example, we may need to create and seed our database before running our flask application.  So let's say that we want to **create our database** always, and then run flask as default.

> Instead of really creating a database we'll just use `echo "create database"` to mock it.

Our thought may be to set up our Dockerfile with the following.

```bash
ENTRYPOINT ["echo", "create database"]

CMD ["flask", "run"]
```

So the thought is to always create the database, and then, with CMD we can *by default* run flask.  But if we build and run this Dockerfile (specified in `web_app/app)`, you'll see the following.

```bash
docker build -t web_app_sh .
```

<img src="./flask-output.png" width="60%">

Ok, so once you call `docker run web_app_sh`, you'll see flask is never booted up.  Instead we just get `create database flask run` echoed to our terminal.  

Do you see why?

Remember that, when used with Entrypoint, whatever we provide in CMD is not separately run, but rather passed through as an additional argument to `echo "create database"`.

### The Fix

We can get around this if we move this `"echo", "create database"` line to a separate initialization script.  Let's see how we can do this.

In the app folder, create a new file called `setup.sh` (the name isn't important, so long as it ends with `.sh`).  And in the `setup.sh` file we can place our initialization lines:

```bash
echo "create database"

exec "$@"
```

And if we want Docker to run this script at runtime, then we change the last four lines of our Dockerfile to the following:

```Dockerfile
RUN pip3 install -r requirements.txt

COPY ./setup.sh .

ENTRYPOINT ["sh", "./setup.sh"]

CMD ["flask", "run"]
```

Ok, so the above will work fine.  But before moving on, let's explain that initialization script.

### Breaking down the Setup script

So currently our setup script looks like the following:

```bash
echo "create database"

exec "$@"
```

Ok, so we are beginning with our usual task of creating the database.  And then have that mysterious `exec "$@"` line.

1. Understanding `exec`

In a bash script the `exec` script will run any command that is passed in as arguments and will not proceed with further with running any additional lines.  Notice, in `web_app` we have provided you with a `practice.sh` script that has the following.

```bash
echo "create database"

exec echo foo

echo "bar"
```

And if you run `sh practice.sh`, notice that `echo foo` is run but `echo bar` is not.

<img src="./practice-app.png" width="60%">

2. Understanding `"$@"`

Ok, so now update the `practice.sh` file to be the following:

```bash
echo "create database"

exec "$@"
```

The `"$@"` means to read in command line arguments passed in when running the file.  For example, run the following:

`sh practice.sh echo hello world`

<img src="./hello-bash.png">

So in this case, we are running the `practice.sh` file, which allows us to see `create database`, and then we are running `echo hello world`, as that's what's being passed through with `sh practice.sh echo hello world`.

### Back to our Dockerfile

So let's take another look at our `Dockerfile` updates.

```Dockerfile
RUN pip3 install -r requirements.txt

COPY ./setup.sh .

ENTRYPOINT ["sh", "./setup.sh"]

CMD ["flask", "run"]
```

And remember `setup.sh` has the following:

```bash
echo "create database"

exec "$@"
```

So above, `flask run` is passed as a default argument to `Entrypoint ["sh", "./setup.sh"]`, meaning that by default we are running: 

* `sh ./setup.sh flask run`

And this is then interpreted by our `setup.sh` file as:

```bash
echo "create database"

flask run
```

And what's nice is that if we provide a arguments when booting up our flask container, these arguments will be executed by the `setup.sh` script.

Try this out, let's run our docker container without any arguments.

```bash
docker build -t web_app_sh .
```

```bash
docker run web_app_sh
```

<img src="./docker-run.png">

And then, we can provide additional arguments, which will over-ride what is specifed in CMD.

```bash
docker run -it web_app_sh sh
```